<a href="https://colab.research.google.com/github/mohamedehab00/A-Hybrid-Arabic-Text-Summarization-Approach-based-on-Transformers/blob/main/MetaDataHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
!pip install transformers

#Essential Imports

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch

# Model

In [ ]:
model_name = "marefa-nlp/marefa-ner"
custom_labels = ["O", "B-job", "I-job", "B-nationality", "B-person", "I-person", "B-location",
                 "B-time", "I-time", "B-event", "I-event", "B-organization", "I-organization",
                 "I-location", "I-nationality", "B-product", "I-product", "B-artwork", "I-artwork"]

NER_tokenizer = AutoTokenizer.from_pretrained(model_name)
NER_Model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(custom_labels))

## Meta Text Extraction Process

In [ ]:
def extractNER(text: str, model: AutoModelForTokenClassification,tokenizer: AutoTokenizer, start_token: str="▁"):
    
    tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt")
    tokenized_sentences = tokenized_sentence['input_ids'].numpy()

    with torch.no_grad():
        output = model(**tokenized_sentence)

    last_hidden_states = output[0].numpy()
    label_indices = np.argmax(last_hidden_states[0], axis=1)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentences[0])
    special_tags = set(tokenizer.special_tokens_map.values())

    grouped_tokens = []
    for token, label_idx in zip(tokens, label_indices):
        if token not in special_tags:
            if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                grouped_tokens[-1]["token"] += token
            else:
                grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

    # extract entities
    ents = []
    prev_label = "O"
    for token in grouped_tokens:
        label = token["label"].replace("I-","").replace("B-","")
        if token["label"] != "O":
            
            if label != prev_label:
                ents.append({"token": [token["token"]], "label": label})
            else:
                ents[-1]["token"].append(token["token"])
            
        prev_label = label
    
    # group tokens
    ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents ]
    
    return ents

# Model Output

In [ ]:
textInput = """
تحت عنوان "أزياء مشعة" قدم المصمم اللبناني بسيل سودة هذا الشهر عرضه في روما،
وهو العرض السابع بعد عدة عروض ناجحة أقامها في بيروت، انتقد سودة هجرة المصممين
العرب إلى الدول الأوروبية لتقديم عروضهم، وطالب بتأسيس أسبوع عربي للموضة
بمواصفات عالمية، يقدم الإبداعات العربية ويعمل على تطوير المهنة، ونفى احتكار
الرجل التصميم للمرأة، مؤكدا أن قدرة المرأة على التصميم للرجل، وآراء أخرى في
هذا الحوار.
 * كيف كان عرضك الأخير في روما وما المجموعة التي قدمتها؟
 ـ قدمت في عرض روما مجموعة من 40 قطعة لموسم الصيف مستوحاة من فترة السبعينيات،
استخدمت فيها الأقمشة التي تستخدم عادة في أزياء الهوت كوتيور مثل الموسلين
والحرير، ولكن بطريقة جديدة مع استخدام الإكسسوار، العرض أقيم تحت عنوان "أزياء
مشعة"، وعلى المستوى العربي كانت الأصداء جيدة جدا.
 * ما هي الألوان والأقمشة التي تفضل استعمالها في تصاميمك؟
 ـ أفضل عادة ألوان الباستيل من الأصفر إلى البيتش سومو والتركواز والبيج والزهر،
وهذه هي ألوان الموضة، وهناك الأسود الذي مازال محافظا على رونقه، والأبيض كذلك،
أما القماش فيحدد حسب موضة العام، فالموضة تحتم عليك نوعية قماش محددة وتضعك في
إطار معين.
 * ما هي المعوقات لمجال تصميم الأزياء في لبنان؟
 ـ المشكلة في عدم وجود أسبوع سنوي للموضة يقام في لبنان على نطاق عالمي، وهجرة
العقول المبدعة، فالمصممون يتركون بلدهم ويعرضون في الخارج، والمفروض أن نقيم
أسبوعا للموضة في العالم العربي بشكل عالمي، فيحدث من خلاله النقد والتطوير، كما
يحدث في روما ولندن ونيويورك اليوم، يجب أن نعمل اليوم قبل الغد على تأسيس هذا
الأسبوع، ولبنان أكثر بلد عنده مقومات إقامة أسبوع موضة، فلديه الأيدي العاملة
والتقنية، مما يمكنه من إقامة أسبوع للموضة، يدرج على المستوى العالمي.
 * لماذا ترتبط الموضة غالبا بارتفاع السعر، وكيف يمكن تحقيق معادلة الأناقة
للجميع؟
 ـ أكيد إن الأناقة حق للجميع، ولكن الهوت كوتيور لا يمكن تقديمه للجميع، تماما
مثل الشخص الذي يشتري منزلا غاليا ويعد له الديكور الخاص به، ولكن عن طريق
الفستان الجاهز والخياطين يمكن أن نقدم تصميمات أنيقة للجميع، ولكن الهوت كوتيور
لا يمكن أن يكون للجميع، لأنها نوعية من الأزياء يتم ارتداؤها في مناسبات معينة
خاصة جدا، مثل الأفراح والحفلات والمناسبات السعيدة، فله ميزات خاصة مكلفة، ويجب
أن نحافظ على هذه الميزات.
 * لماذا يحتكر الرجل مهنة التصميم للمرأة؟
 ـ في العالم توجد مصممات كثيرات نجحن في التصميم للمرأة، ولكن في العالم العربي
يندر ذلك لظروف اجتماعية مختلفة، أيضا المرأة لو صممت تصميما جيدا بفطرتها ستقول
هذا التصميم لي، ليس في الأمر احتكار، بالعكس نحن لا نرفض الفكرة فالمرأة قادرة
على الإبداع، والرجولة لا تحتكر تصميم الأزياء.
 * صممت إحدى الشركات اليابانية روبوت خاصاً لعرض الأزياء، فهل تنجح هذه الفكرة؟.
 ـ الموديل يجب أن تعرضه امرأة، ولا أتصور أن يتم بخلاف ذلك، فلا يمكن أن أعرض
تصميماتي على روبوت، هذه تكنولوجيا ولا أتصور تطبيقها في الهوت كوتيور، هناك
محاولات تمت من هذا القبيل، مثال عرض سينمائي أقيم في التسعينيات نفذه مصمم فرنسي
اسمه هيري موجلير، ونجح العرض ولكن كان مكلفا، فقد كلف مليون دولار، وكانت محاولة
في فترة تحول للعروض السينمائية، وهناك محاولات أخرى سابقة تمت، ولكنها كانت
بمثابة رسائل وتنبيه إلى أن الإنسان تحول إلى روبوت، وأننا ذاهبون إلى الميكنة،
ولكن لا يمكن أن نستعيض عن المرأة أو الرجل بروبوت.
 * كيف تأتي فكرة التصميم؟
 ـ أحيانا يكون كالإلهام فقد يأتي فجأة، وقد يكون نتيجة بحث، بعد أن تضع نفسك في
جو معين من الموسيقي والسينما والنحت والرسم، فهو إنتاج فني متكامل، تظهر معه
أشياء من الجو الذي تعيش فيه، أو تحب أن تكون فيه، إنها حالة كالحلم، ترسم موديل
ثم تنفذه أو تنفذه مباشرة، وذلك حسب الفكرة وكيفية تطورها.
 * ما رأيك ببعض المصممين الذين تمردوا على السمات الأساسية لفستان الزفاف اللون
الأبيض؟ 
ـ تطوير فستان الفرح ممكن، ولكن اللون الأبيض أساسي، فلا يمكن أن نصممه أسود أو
أحمر، يمكن أن يكون لونه رمادياً فاتحاً أو أبيض أو ذهبياً، ولكن عدا ذلك لا
أعتقد، ومن الصعب وجود مصمم يتخطى هذه الحدود، وإذا وجدت هذه الألوان في العروض
فإن 99% من العرائس لا يرتدونه، ولكن هذا لا يمنع أن يكون جميلا.
 * ما دور الإكسسوارات في تصاميمك؟
 ـ ركزت على الإكسسوارات في بعض موديلات هذه المجموعة، وفي المجموعة السابقة قدمت
فساتين مصنوعة من الإكسسوار، ولكن بطريقتي الخاصة، وكانت مستوحاة من الجو الماسي،
وأنا أفضل مجوهرات الكريستال، وأن يكون اللوك العام للإكسسوارات مجوهرات.
 * كيف يمكن الارتقاء بمهنة التصميم؟
 ـ على الدول العربية تشجيع المهنة لكي نتطور ونصل إلى المستوى العالمي، كل دور
الأزياء مرتبطة لدينا بأشخاص، في الخارج دور الأزياء كيانات كبيرة، المصمم ليس
صاحبها، وتوجد فيها هيكلية في الإدارة، وهذا قليل في العالم العربي، وإذا وجد
لدينا سيساعد المصمم كثيرا على التصميم، بعيدا عن التسويق والإدارة، ولم نصل بعد
إلى هذه المرحلة.
 في الوطن العربي هناك معدل إنفاق عال على الأزياء، ولكن لا يوجد تطور، ومهنة
الهوت كوتيور صناعة آخذة في التراجع، وفي أوروبا أصبحوا يستخدمونها في الإعلام
وفي تسويق أسمائهم، لا يهمهم أن يبيعوا، ومن خلالها يتم تسويق مواد أخرى تحت هذا
الاسم، فهم ينفقون ملايين الدولارات على العروض، ولكن تحت إطار تسويقي وإعلامي،
وعندنا يتهم المصممون العرب بأن ليس لديهم إبداع، إما أن تبدع أو تبيع..
"""

In [ ]:
extractNER(text=textInput, model=NER_Model, tokenizer=NER_tokenizer, start_token="▁")

[{'label': 'job', 'token': 'المصمم'},
 {'label': 'nationality', 'token': 'اللبناني'},
 {'label': 'person', 'token': 'بسيل سودة'},
 {'label': 'location', 'token': 'روما،'},
 {'label': 'location', 'token': 'بيروت،'},
 {'label': 'person', 'token': 'سودة'},
 {'label': 'job', 'token': 'المصممين'},
 {'label': 'location', 'token': 'الدول الأوروبية'},
 {'label': 'location', 'token': 'روما'},
 {'label': 'location', 'token': 'روما'},
 {'label': 'time', 'token': 'السبعينيات،'},
 {'label': 'artwork', 'token': 'الهوت كوتيور'},
 {'label': 'location', 'token': 'لبنان؟'},
 {'label': 'location', 'token': 'لبنان'},
 {'label': 'location', 'token': 'العالم العربي'},
 {'label': 'location', 'token': 'روما ولندن ونيويورك'},
 {'label': 'artwork', 'token': 'الهوت كوتيور'}]